Alex Thornton (apt2141)  
Columbia University Fall 2021  
ELEN E6893 - Big Data Analytics  
Research Project: SpotifyClassifier

1. Connect to GCP

In [ ]:
from google.colab import auth
auth.authenticate_user()
!gcloud config set project spotifyclassifier
!pip install spotipy

Updated property [core/project].


In [ ]:
from googleapiclient.discovery import build
gcs_service = build('storage', 'v1')
from apiclient.http import MediaIoBaseDownload
from googleapiclient.http import MediaFileUpload

3. Import Packages

In [ ]:
import requests
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

3. Setup Spotify API

In [ ]:
auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)

4. Do Stuff

In [ ]:
genres = ["alternative","indie","alt-rock","punk","grunge","punk-rock","indie-pop","emo","goth",\
    "hardcore","hardcore","death-metal","grindcore","black-metal","heavy-metal","metalcore","metal","metal-misc",\
    "dance","electronic","electro","edm","garage","house","chicago-house","breakbeat","dubstep","dance","techno","j-dance","deep-house",\
    "ambient","chill","sleep","new-age","study","piano",\
    "rock","rock-n-roll","rockabilly","j-rock","hard-rock","psych-rock","power-pop","guitar","drum-and-bass",\
    "country","bluegrass","acoustic","folk","honky-tonk",\
    "pop","k-pop","j-pop","cantopop","party","pop-film","r-n-b","club",\
    "jazz","soul","funk","blues","afrobeat",\
    "latin","latino","reggaeton","samba","salsa","bossanova",\
    "classical","opera",\
    "show-tunes","disney"]

!mkdir /track-files
for genre in genres:
    with open('/track-files/'+genre+'.json', 'wb') as f:
        request = gcs_service.objects().get_media(bucket='track-lists-full',object=genre+'.json')
        media = MediaIoBaseDownload(f, request)
        done = False
        while not done:
            _, done = media.next_chunk()

    with open('/track-files/'+genre+'.json', "r") as tracklist:
        tracklist = json.load(tracklist)
        num_songs = len(tracklist.keys())
    print("Genre: "+genre+", # Tracks: "+str(num_songs))



Genre: ambient, # Tracks: 966
Genre: alternative, # Tracks: 975
Genre: anime, # Tracks: 947
Genre: blues, # Tracks: 921
Genre: classical, # Tracks: 516
Genre: country, # Tracks: 962
Genre: death-metal, # Tracks: 971
Genre: dubstep, # Tracks: 977
Genre: electronic, # Tracks: 969
Genre: folk, # Tracks: 963
Genre: gospel, # Tracks: 926
Genre: goth, # Tracks: 933
Genre: grindcore, # Tracks: 963
Genre: guitar, # Tracks: 926
Genre: hip-hop, # Tracks: 967
Genre: honky-tonk, # Tracks: 716
Genre: indie, # Tracks: 970
Genre: jazz, # Tracks: 552
Genre: k-pop, # Tracks: 581
Genre: latin, # Tracks: 970
Genre: new-age, # Tracks: 959
Genre: opera, # Tracks: 929
Genre: punk, # Tracks: 863
Genre: show-tunes, # Tracks: 893
Genre: singer-songwriter, # Tracks: 968


In [ ]:
!ls /track-files

alternative.json  dubstep.json	   hip-hop.json     opera.json
ambient.json	  electronic.json  honky-tonk.json  punk.json
anime.json	  folk.json	   indie.json	    show-tunes.json
blues.json	  gospel.json	   jazz.json	    singer-songwriter.json
classical.json	  goth.json	   k-pop.json
country.json	  grindcore.json   latin.json
death-metal.json  guitar.json	   new-age.json


In [ ]:
!mkdir /audio-features

mkdir: cannot create directory ‘/audio-features’: File exists


In [ ]:
for genre in genres:
    !mkdir /audio-features/$genre

mkdir: cannot create directory ‘/audio-features/alternative’: File exists


In [ ]:
for genre in genres:
    with open('/track-files/'+genre+'.json', "r") as tracklist:
        tracklist = json.load(tracklist)
        for track in tracklist:
            track_id = tracklist[track]
            track_data = sp.audio_features(track_id)

            # Save audio features locally
            with open("/audio-features/"+track_id+".json", 'w') as f:
                #Add some of our own labels
                track_data['subgenre-label'] = genre 
                track_data['super-genre-label'] = supergenre[genre]
                track_data['song-title'] = track

                #Fetch some additional info for us
                basic_info = sp.track(track_id)
                track_data['popularity'] = basic_info['popularity']
                track_data['explicit'] = basic_info['explicit']
                album = basic_info['album']
                track_data['release_date'] = album['release_date']
                track_data['release_date_precision'] = album['release_date_precision']

                #Add in some Spotify Audio Analysis Vectors
                analysis = sp.audio_analysis(id)

                num_seg = 0
                pitches = np.zeros(12)
                timbre = np.zeros(12)

                analysis = analysis['track']
                time_signature = analysis['time_signature']
                key = analysis['key']
                mode = analysis['mode']

                if "segments" in analysis:
                    for _, j in enumerate(analysis['segments']):
                        pitches += np.array(j['pitches'])
                        timbre += np.array(j['timbre'])
                        num_seg+=1

                track_data['avg_pitches'] = pitches/num_seg
                track_data['avg_timbre'] = timbre/num_seg

                json.dump(track_data, f, indent=4)

            # Upload audio features to GCP bucket
            media = MediaFileUpload("/audio-features/"+track_id+".json", 
                              mimetype='application/json',
                              resumable=True)

            request = gcs_service.objects().insert(bucket='audio-features-fullset', 
                                                  name=genre+"/"+track_id+".json",
                                                  media_body=media)

            response = None
            while response is None:
              _, response = request.next_chunk()

      



KeyboardInterrupt: ignored